In [19]:
import praw
import datetime as dt
import time
import numpy as np
import pandas as pd
import math
import sys

NUMPOSTS = 25
reddit = praw.Reddit('investor')
#early: 20-30 minutes old
#mid: 60-75 minutes old
#mature: >5 hrs old
#numComments: early comments number
columns = ['title', 'earlyScore', 'midScore', 'numComments','matureScore']
df = pd.DataFrame(columns=columns)

subreddit = reddit.subreddit("memeeconomy")
ct = 0
def get_date(sub):
    time = sub.created
    return dt.datetime.fromtimestamp(time)

print(str(dt.datetime.now()))

2018-08-25 21:32:49.742883


This thing down here collects data from r/memeeconomy. It looks for 20 minute old posts, hour old posts, and 4 hour old posts, and sees how many upvotes they have. I'm pretty sure there's a really strong correlation between upvotes at 20 minutes and final upvotes. 

It turns out that I was right, and that if a post has more than ~30 upvotes at 20 minutes, it is basically guaranteed to have ~140% return. 

But, looking at the data I collected, it seems like 0 upvote posts might be more promising. The formula 

In [ ]:
while dt.datetime.now().hour != 22:
    for submission in subreddit.new(limit=100):
        createdTime = submission.created - 8 * 3600 #to account for time difference
        ageMinutes = (-createdTime + time.mktime(dt.datetime.now().timetuple()))/60

        #print('dateTime: ' + str(get_date(submission)) + '\n diff (mins): ' + str(ageMinutes) + '\n title: ' + str(submission.title))
        #print('score: ' + str(submission.score)) 

        if (ageMinutes < 30) & (ageMinutes > 20):
            #print('right range')
            t = submission.title
            t5 = t[:5]
            df.loc[submission.id] = [t5, submission.score,-0.5,-0.5,-0.5]
           # np.append(data, [submission.id, submission.score, 0, 0, 0])
            #print(df)

        if (ageMinutes < 75) & (ageMinutes > 60):
            df.at[submission.id, 'midScore'] = submission.score

        if (ageMinutes > 5 * 60 ) & (ageMinutes < 5*60 + 3):
            df.at[submission.id, 'matureScore'] = submission.score

j = df.to_json(orient='records')
with open(str(dt.datetime.now()) + '_posts.json', 'w') as f:
    f.write(j)


In [34]:
bal = 1466

In [33]:
#main
investedPosts =['']
ct = 0
while True:
    for submission in subreddit.new(limit=20):
        createdTime = submission.created - 8 * 3600 #to account for time difference
        ageMinutes = (-createdTime + time.mktime(dt.datetime.now().timetuple()))/60
        
        #print('dateTime: ' + str(get_date(submission)) + '\n diff (mins): ' + str(ageMinutes) + '\n title: ' + str(submission.title))
        #print('score: ' + str(submission.score)) 
        #print(ageMinutes)
        if (ageMinutes < 25) & (ageMinutes > 20) & (submission.score > 30) & (submission.id not in investedPosts):
            #print('right range')
            for topLevelComment in submission.comments:
                print(topLevelComment.author)
                if topLevelComment.author == 'MemeInvestor_bot':
                    print('gotem ' + submission.title)
                    topLevelComment.reply('!invest 100')
                    break
                    
            investedPosts.append(submission.id)
        

MemeInvestor_bot
gotem INVEST NOW! Great template with high potential for variability.
gotem INVEST NOW! Great template with high potential for variability.
MemeInvestor_bot
gotem Very versatile. Invest now!
gotem Very versatile. Invest now!
MemeInvestor_bot
gotem Keanu always returns high yields. After appraisal.
gotem Keanu always returns high yields. After appraisal.
MemeInvestor_bot
gotem Are behemoth memes in???
gotem Are behemoth memes in???
MemeInvestor_bot
gotem Any potential for a new format?


SystemExit: 0

/home/yuwen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
df